In [1]:
import os
from os.path import abspath
import findspark
import json
import requests
findspark.init()

In [2]:
os.environ["JAVA_HOME"] = r"/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_LOCAL_IP"] = "172.22.24.52"
os.environ["SPARK_HOME"] = r"/home/guilherme/spark-3.4.0-bin-hadoop3"
os.environ["LIVY_CONF_DIR"] = r"/home/guilherme/apache-livy-0.7.1-incubating-bin"

In [3]:
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark import SparkContext, SparkConf
import tempfile

warehouse_location = abspath('spark-warehouse')

In [ ]:
conf = SparkConf()
conf.set("spark.master","local[*]")
conf.set("spark.submit.deployMode","client")
conf.set("spark.executor.memory", "4g")
conf.set("spark.driver.memory", "4g")
conf.set("spark.yarn.am.memory","1g")
conf.set("spark.eventLog.enabled ","true")
conf.set("spark.sql.adaptive.enabled","true")
spark = SparkSession.builder\
    .config(conf=conf)\
    .config("spark.sql.warehouse.dir", warehouse_location)\
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()

In [6]:
import json, pprint, requests, textwrap
host = 'http://localhost:8998'
data = {'kind': 'spark'}
headers = {'Content-Type': 'application/json'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

{u'state': u'starting', u'id': 0, u'kind': u'spark'}

{'state': 'starting', 'id': 0, 'kind': 'spark'}

In [7]:
data = {'kind': 'pyspark'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

{u'id': 1, u'state': u'idle'}

{'id': 1, 'state': 'idle'}

In [8]:
session_url = 'http://localhost:8998/sessions/0'
requests.delete(session_url, headers=headers)

<Response [404]>

In [9]:

r = requests.get(session_url)
pprint.pprint(r.json())

{u'id': 1,
 u'output': {u'data': {u'text/plain': u'Pi is roughly 3.14004\nNUM_SAMPLES: Int = 100000\ncount: Int = 78501'},
             u'execution_count': 1,
             u'status': u'ok'},
 u'state': u'available'}

{'msg': "Session '0' not found."}


{'id': 1,
 'output': {'data': {'text/plain': 'Pi is roughly 3.14004\nNUM_SAMPLES: Int = 100000\ncount: Int = 78501'},
  'execution_count': 1,
  'status': 'ok'},
 'state': 'available'}

In [10]:
data = {'kind': 'sparkr'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

{u'id': 1, u'state': u'idle'}

{'id': 1, 'state': 'idle'}

In [11]:

url = "http://localhost:8998/sessions"

payload = {}
headers = {
  'X-Requested-By': 'admin'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


{"from":0,"total":6,"sessions":[{"id":23,"name":null,"appId":null,"owner":null,"proxyUser":null,"state":"dead","kind":"spark","appInfo":{"driverLogUrl":null,"sparkUiUrl":null},"log":["\tat org.apache.spark.deploy.SparkSubmit.submit(SparkSubmit.scala:215)","\tat org.apache.spark.deploy.SparkSubmit.doSubmit(SparkSubmit.scala:91)","\tat org.apache.spark.deploy.SparkSubmit$$anon$2.doSubmit(SparkSubmit.scala:1111)","\tat org.apache.spark.deploy.SparkSubmit$.main(SparkSubmit.scala:1120)","\tat org.apache.spark.deploy.SparkSubmit.main(SparkSubmit.scala)","Caused by: java.lang.ClassNotFoundException: scala.Function0$class","\tat java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)","\tat java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:589)","\tat java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:522)","\t... 20 more"]},{"id":24,"name":null,"appId":null,"owner":null,"proxyUser":null,"state":"dead","kind":"pyspark","appInfo":{"driverLogUrl":null,"sparkUiUrl":n

In [ ]:
url = "http://localhost:8998/batches"

payload = json.dumps({
  "file": "hdfs://10.250.36.25:9000/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibs/o2ov2.coding.category-1.0.0-PUSH.jar",
  "className": "com.nielseniq.o2ov2.catecoding.O2OV2CategoryCodingSplitApp",
  "jars": [
    "hdfs://10.250.36.25:9000/projects/O2OV2/APP_PACKAGE/DEV/O2OV2CommonLibs/*",
    "hdfs://10.250.36.25:9000/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibs/CommonInfra-2.10.0-O2OV2.jar",
    "hdfs://10.250.36.25:9000/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibs/o2ov2.common-1.0.0.jar"
  ],
  "name": "DEV_O2OV2_CategoryCoding_PUSH_20221404",
  "driverMemory": "4g",
  "executorMemory": "4g",
  "numExecutors": 7,
  "executorCores": 4
})
headers = {
  'X-Requested-By': 'admin',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)